In [1]:
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import unicodedata

In [2]:
# took Adi Bronshtein's function for conducting exploratory data analysis and added spaces between each eda test
# using blank print statesments and \n for easier readability
def eda(dataframe):
    print("missing values: \n{}".format(dataframe.isnull().sum()))
    print()
    print("dataframe index: \n{}".format(dataframe.index))
    print()
    print("dataframe types: \n{}".format(dataframe.dtypes))
    print()
    print("dataframe shape: \n{}".format(dataframe.shape))
    print()
    print("dataframe describe: \n{}".format(dataframe.describe()))
    print()

    print("number of unique values in each column: ")
    for item in dataframe:
        print(item)
        print(dataframe[item].nunique())

In [3]:
# function to strip strings of accents, from BartoszKP and oefe on StackOverflow
# https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-in-a-python-unicode-string

import unicodedata
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')


In [4]:
def has_same_cheese(df1, df2):
    count = 0

    for i in range(len(df1)):
        for k in range(len(df2)):
            if strip_accents(df2['Name'][k].lower()) in strip_accents(df1['Name'][i].lower()):
            #if df1['Name'][i] == df2['Name'][k]:
                count += 1
                print(df1['Name'][i])
                print(df2['Name'][k])
                print()
                
    print(f"The first dataset had {count} cheeses that were in the second dataset.")

In [5]:

def dummy_listed_features(df, feature):

    # for ceratin features, some cells have multiple values
    # to get all of the values of these features, first create a list 
    # of all of the UNIQUE values that appear in the selected feature

    # fill the null values of the feature with a string value of "None" so as to not throw an error
    df[feature].fillna('None', inplace=True)

    # create an empty string
    strang = ""
    # look through every cell of the feature
    for i in list(df[feature].unique()):
        # add every entry in the cell into a long string seperated by a comma and a space
        strang += i + ', '

    # strip the extraneous comma and whitespace from the end of the newly gathered string
    # replace ' and' with a comma
    strang = strang.replace(' and', ',').strip(', ')

    # seperate the long string of values by a comma and space into a list of values in the feature
    # convert this list into a set to drop the values that appear more then once in an entry for the feature
    # convert this back into a list so it can be indexed 
    feature_list = list(set(strang.split(', ')))

    # initialize a dummy column for each unique value of the feature
    for i in feature_list:
        # initialize each column as 0
        df[f'{feature}_{i}']=0

    # go through every cheese in the list
    for i in range(len(df)):

        # look at each value in each cell by seperating by commas, putting it into a list, 
        # and iterating through that list (remember to replace any ' and' with a comma before creating list)
        for k in df[feature][i].replace(' and', ',').split(', '):
            # if the value exists in the feature for a particular cheese, set the dummy value to 1
            df[f'{feature}_{k}'][i] = 1

            
            
    # once feature has been dummied, drop the original        
    df.drop(columns=feature, inplace=True)

    # also drop the feature column created when the value was 'None' because that was a null value before        
    df.drop(columns=f'{feature}_None', inplace=True)

In [6]:
#read model_testing.csv into 'cheese_com'
cheese_com = pd.read_csv('./data/model_testing.csv')

In [7]:
#read culture_cheese_mag.csv into 'culture_chz'
culture_chz = pd.read_csv('./data/culture_model_testing.csv')

In [13]:
#read canadianCheeseDirectory.csv into 'can_chz'
#can_chz = pd.read_csv('./data/canadian_cheese_part_clean.csv')

In [8]:
#read fork_and_bottle.csv into 'chz_fork'
#chz_fork = pd.read_csv('./data/fork_and_bottle.csv')


In [8]:
# Merge DataFrames
df = pd.merge(cheese_com, culture_chz, on= 'Name', how ='outer')

In [11]:
df

,Name,Milk_x,Country of origin,Region_x,Type,Texture,Rind_x,Flavour,Aroma,Vegetarian,...,Region_y,Milk_y,Treatment,Classification,Rennet,Rind_y,Style,Flavor,Description_y,Smoked
0,Abbaye de Belloc,unpasteurized cow's and sheep's milk,France,Pays Basque,"semi-hard, artisan","creamy, dense and firm",natural,burnt caramel,lanoline,yes,...,Aquitaine,Sheep,Raw,Firm,Animal,"Natural, Paprika","Alpine-style, Cheddar-like",Flavor added to rind,Abbaye de Belloc is a French cheese that is ma...,0.0
1,Abbaye de Belval,cow's milk,France,NaN,semi-hard,elastic,washed,NaN,aromatic,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Abbaye de Citeaux,unpasteurized cow's milk,France,Burgundy,"semi-soft, artisan, brined","creamy, dense and smooth",washed,"acidic, milky, smooth","barnyardy, earthy",no,...,Burgundy,Cow,Raw,Semi Soft,Animal,"Washed, Mold Ripened","Soft-Ripened (Brie-like), Washed-Rind",Unflavored,"Located near Losne in Burgundy, France, Abbaye...",0.0
3,Abbaye de Timadeuc,pasteurized cow's milk,France,province of Brittany,semi-hard,soft,washed,NaN,NaN,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Abbaye du Mont des Cats,pasteurized cow's milk,France,Nord-Pas-de-Calais,"semi-soft, artisan, brined",smooth and supple,washed,"milky, salty",floral,no,...,Nord-Pas-de-Calais,Cow,Pasteurised,Semi Soft,Vegetable,Washed,"Alpine-style, Washed-Rind",Unflavored,Abbaye du Mont des Cats is a cow`s milk cheese...,0.0
5,Abbot’s Gold,pasteurized cow's milk,"England, Great Britain and United Kingdom",North Yorkshire,semi-hard,"creamy, crumbly, dense and semi firm",natural,"mild, sweet, tangy",aromatic,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Abertam,sheep's milk,Czech Republic,Karlovy Vary,"hard, artisan",firm,natural,"acidic, strong, tangy",NaN,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Abondance,unpasteurized cow's milk,France,"Haute-Savoie, Abondance","semi-hard, artisan","creamy, open and supple",natural,"acidic, buttery, fruity, sweet",nutty,no,...,Haut Savoie,Cow,Raw,Firm,Animal,Natural,Alpine-style,Unflavored,Named after a valley in the French Alps where ...,0.0
8,Acapella,goat's milk,United States,California,"soft, soft-ripened",NaN,NaN,buttery,"fresh, herbal",no,...,California,Goat,Pasteurised,Soft,Microbial,"Mold Ripened, Ash coated",Soft-Ripened (Brie-like),Unflavored,"Established in 1999, Andante Dairy is owned an...",0.0
9,Accasciato,Buffalo's and cow's milk,Italy,Campania,semi-hard,firm,natural,sweet,"aromatic, fresh",no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df.sort_values('Name').reset_index(drop=True)

,Name,Milk_x,Country of origin,Region_x,Type,Texture,Rind_x,Flavour,Aroma,Vegetarian,...,Region_y,Milk_y,Treatment,Classification,Rennet,Rind_y,Style,Flavor,Description_y,Smoked
0,ADL Brick Cheese,pasteurized cow's milk,Canada,Prince Edward Island,semi-soft,"elastic, firm, open and soft",rindless,"buttery, mild, milky, subtle","buttery, sweet",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ADL Mild Cheddar,pasteurized cow's milk,Canada,Prince Edward Island,semi-hard,firm and springy,rindless,"acidic, buttery, milky, subtle",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Abbaye de Belloc,unpasteurized cow's and sheep's milk,France,Pays Basque,"semi-hard, artisan","creamy, dense and firm",natural,burnt caramel,lanoline,yes,...,Aquitaine,Sheep,Raw,Firm,Animal,"Natural, Paprika","Alpine-style, Cheddar-like",Flavor added to rind,Abbaye de Belloc is a French cheese that is ma...,0.0
3,Abbaye de Belval,cow's milk,France,NaN,semi-hard,elastic,washed,NaN,aromatic,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Abbaye de Citeaux,unpasteurized cow's milk,France,Burgundy,"semi-soft, artisan, brined","creamy, dense and smooth",washed,"acidic, milky, smooth","barnyardy, earthy",no,...,Burgundy,Cow,Raw,Semi Soft,Animal,"Washed, Mold Ripened","Soft-Ripened (Brie-like), Washed-Rind",Unflavored,"Located near Losne in Burgundy, France, Abbaye...",0.0
5,Abbaye de Tamie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Savoie,Cow,Raw,Semi Soft,Animal,Washed,Washed-Rind,Unflavored,"Made from raw cow`s milk, Abbaye de Tamie is p...",0.0
6,Abbaye de Timadeuc,pasteurized cow's milk,France,province of Brittany,semi-hard,soft,washed,NaN,NaN,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Abbaye du Mont des Cats,pasteurized cow's milk,France,Nord-Pas-de-Calais,"semi-soft, artisan, brined",smooth and supple,washed,"milky, salty",floral,no,...,Nord-Pas-de-Calais,Cow,Pasteurised,Semi Soft,Vegetable,Washed,"Alpine-style, Washed-Rind",Unflavored,Abbaye du Mont des Cats is a cow`s milk cheese...,0.0
8,Abbot’s Gold,pasteurized cow's milk,"England, Great Britain and United Kingdom",North Yorkshire,semi-hard,"creamy, crumbly, dense and semi firm",natural,"mild, sweet, tangy",aromatic,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Abertam,sheep's milk,Czech Republic,Karlovy Vary,"hard, artisan",firm,natural,"acidic, strong, tangy",NaN,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df2 = pd.merge(cheese_com, culture_chz, on= 'Name', how ='left')

In [15]:
df2

,Name,Milk_x,Country of origin,Region_x,Type,Texture,Rind_x,Flavour,Aroma,Vegetarian,...,Region_y,Milk_y,Treatment,Classification,Rennet,Rind_y,Style,Flavor,Description_y,Smoked
0,Abbaye de Belloc,unpasteurized cow's and sheep's milk,France,Pays Basque,"semi-hard, artisan","creamy, dense and firm",natural,burnt caramel,lanoline,yes,...,Aquitaine,Sheep,Raw,Firm,Animal,"Natural, Paprika","Alpine-style, Cheddar-like",Flavor added to rind,Abbaye de Belloc is a French cheese that is ma...,0.0
1,Abbaye de Belval,cow's milk,France,NaN,semi-hard,elastic,washed,NaN,aromatic,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Abbaye de Citeaux,unpasteurized cow's milk,France,Burgundy,"semi-soft, artisan, brined","creamy, dense and smooth",washed,"acidic, milky, smooth","barnyardy, earthy",no,...,Burgundy,Cow,Raw,Semi Soft,Animal,"Washed, Mold Ripened","Soft-Ripened (Brie-like), Washed-Rind",Unflavored,"Located near Losne in Burgundy, France, Abbaye...",0.0
3,Abbaye de Timadeuc,pasteurized cow's milk,France,province of Brittany,semi-hard,soft,washed,NaN,NaN,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Abbaye du Mont des Cats,pasteurized cow's milk,France,Nord-Pas-de-Calais,"semi-soft, artisan, brined",smooth and supple,washed,"milky, salty",floral,no,...,Nord-Pas-de-Calais,Cow,Pasteurised,Semi Soft,Vegetable,Washed,"Alpine-style, Washed-Rind",Unflavored,Abbaye du Mont des Cats is a cow`s milk cheese...,0.0
5,Abbot’s Gold,pasteurized cow's milk,"England, Great Britain and United Kingdom",North Yorkshire,semi-hard,"creamy, crumbly, dense and semi firm",natural,"mild, sweet, tangy",aromatic,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Abertam,sheep's milk,Czech Republic,Karlovy Vary,"hard, artisan",firm,natural,"acidic, strong, tangy",NaN,no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Abondance,unpasteurized cow's milk,France,"Haute-Savoie, Abondance","semi-hard, artisan","creamy, open and supple",natural,"acidic, buttery, fruity, sweet",nutty,no,...,Haut Savoie,Cow,Raw,Firm,Animal,Natural,Alpine-style,Unflavored,Named after a valley in the French Alps where ...,0.0
8,Acapella,goat's milk,United States,California,"soft, soft-ripened",NaN,NaN,buttery,"fresh, herbal",no,...,California,Goat,Pasteurised,Soft,Microbial,"Mold Ripened, Ash coated",Soft-Ripened (Brie-like),Unflavored,"Established in 1999, Andante Dairy is owned an...",0.0
9,Accasciato,Buffalo's and cow's milk,Italy,Campania,semi-hard,firm,natural,sweet,"aromatic, fresh",no,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
df2 = pd.merge(df, can_chz, on= 'Name', how ='outer')

In [14]:
can_chz

,Name,Manufacturer,Provence,Source_Type,FatContentPercent,MoisturePercent,Rando_Details,Flavor,Characteristics,Age,Organic,Category,Animal,Raw_or_Past,Rind
0,Sieur de Duplessis (Le),Fromages la faim de loup,NB,Farmstead,24.2,47.0,NaN,"Sharp, lactic",Uncooked,9 Months,0,Firm Cheese,Ewe,Raw Milk,Washed Rind
1,Tomme Le Champ Doré,Fromages la faim de loup,NB,Farmstead,24.2,47.9,NaN,"Sharp, lactic, lightly caramelized",Uncooked,NaN,0,Semi-soft Cheese,Cow,Raw Milk,Washed Rind
2,Provolone Sette Fette (Tre-Stelle),Tre Stelle (Arla Foods),ON,Industrial,24.0,54.0,NaN,"Mild, tangy, and fruity","Pressed and cooked cheese, pasta filata, inter...",NaN,0,Firm Cheese,Cow,Pasteurized,NaN
3,Geai Bleu (Le),Fromages la faim de loup,NB,Farmstead,29.0,47.0,NaN,Sharp with fruity notes and a hint of wild honey,NaN,3 Months,0,Veined Cheeses,Cow,Raw Milk,NaN
4,Gamin (Le),Fromages la faim de loup,NB,Farmstead,24.6,49.4,NaN,Softer taste,NaN,2 Months,1,Semi-soft Cheese,Cow,Raw Milk,Washed Rind
5,Paneer (Northumberland Co-operative),Northumberland Co-operative,NB,Industrial,25.0,48.0,"Traditional Indian Cheese, also available : Li...",NaN,Classic fresh cooking cheeses,Less than 1 Month,0,Fresh Cheese,Cow,Pasteurized,NaN
6,Goat Brie (Woolwich),Woolwich Dairy,ON,Industrial,22.0,52.0,"Rennet free, ripens from the outside in","Rich, creamy, buttery, both subtle and tangy i...",NaN,Less than 1 Month,0,Soft Cheese,Goat,Pasteurized,Bloomy Rind
7,Goat Cheddar (Woolwich),Woolwich Dairy,ON,Industrial,29.0,41.0,"Medium Aged, rennet free",Mild,"Whitem, smooth, firm textured",4 Months,0,Firm Cheese,Goat,Pasteurized,NaN
8,Goat Mozarella (Woolwich),Woolwich Dairy,ON,Industrial,22.0,50.0,Rennet free,Mild,NaN,NaN,0,Semi-soft Cheese,Goat,Pasteurized,NaN
9,Goat Feta (Woolwich),Woolwich Dairy,ON,Industrial,22.0,55.0,Rennet free,"Sharp, tangy, salty",With or without brine,Less than 1 Month,0,Soft Cheese,Goat,Pasteurized,NaN


In [59]:
df3 = pd.merge(df2, chz_fork, on= 'Name', how ='outer')

In [60]:
df3.columns

Index(['Name', 'Milk_x', 'Country of origin', 'Region_x', 'Type', 'Texture',
       'Rind_x', 'Colour', 'Flavour', 'Aroma', 'Vegetarian', 'Producers',
       'Synonyms', 'Description_x', 'Fat content', 'Family',
       'Alternative spellings', 'Calcium content',
       'Fat content (in dry matter)', 'Producer', 'Country', 'Region_y',
       'Size', 'Weight', 'Website', 'Milk_y', 'Treatment', 'Classification',
       'Rennet', 'Rind_y', 'Style', 'Flavor_x', 'Description_y',
       'Manufacturer', 'Provence', 'Source_Type', 'FatContentPercent',
       'MoisturePercent', 'Rando_Details', 'Flavor_y', 'Characteristics',
       'Age', 'Organic', 'Category', 'Animal', 'Raw_or_Past', 'Rind', 'Milk',
       'Region', 'Description'],
      dtype='object')

In [61]:
df3

,Name,Milk_x,Country of origin,Region_x,Type,Texture,Rind_x,Colour,Flavour,Aroma,...,Characteristics,Age,Organic,Category,Animal,Raw_or_Past,Rind,Milk,Region,Description
0,Abbaye de Belloc,unpasteurized cow's and sheep's milk,France,Pays Basque,"semi-hard, artisan","creamy, dense and firm",natural,yellow,burnt caramel,lanoline,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Abbaye de Belval,cow's milk,France,NaN,semi-hard,elastic,washed,ivory,NaN,aromatic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Abbaye de Citeaux,unpasteurized cow's milk,France,Burgundy,"semi-soft, artisan, brined","creamy, dense and smooth",washed,white,"acidic, milky, smooth","barnyardy, earthy",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Abbaye de Timadeuc,pasteurized cow's milk,France,province of Brittany,semi-hard,soft,washed,pale yellow,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Abbaye du Mont des Cats,pasteurized cow's milk,France,Nord-Pas-de-Calais,"semi-soft, artisan, brined",smooth and supple,washed,pale yellow,"milky, salty",floral,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Abbot’s Gold,pasteurized cow's milk,"England, Great Britain and United Kingdom",North Yorkshire,semi-hard,"creamy, crumbly, dense and semi firm",natural,pale yellow,"mild, sweet, tangy",aromatic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Abertam,sheep's milk,Czech Republic,Karlovy Vary,"hard, artisan",firm,natural,pale yellow,"acidic, strong, tangy",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Abondance,unpasteurized cow's milk,France,"Haute-Savoie, Abondance","semi-hard, artisan","creamy, open and supple",natural,pale yellow,"acidic, buttery, fruity, sweet",nutty,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Acapella,goat's milk,United States,California,"soft, soft-ripened",NaN,NaN,NaN,buttery,"fresh, herbal",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Accasciato,Buffalo's and cow's milk,Italy,Campania,semi-hard,firm,natural,pale yellow,sweet,"aromatic, fresh",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Name,Producer,Country,Region,Size,Weight,Website,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description
0,Abbaye de Belloc,Abbaye de Notre-Dame de Belloc,France,Aquitaine,"8-9 in. diameter, 3-4 in. high",8-10 lbs,NaN,Sheep,Raw,Firm,Animal,NaturalPaprika,Alpine-styleCheddar-like,Flavor added to rind,Abbaye de Belloc is a French cheese that is ma...
1,Abbaye de Citeaux,Frère Joel & Frère Frédéric,France,Burgundy,8 in. diameter. 2 in. high,2 lbs,NaN,Cow,Raw,Semi Soft,Animal,WashedMold Ripened,Soft-Ripened (Brie-like)Washed-Rind,NaN,"Located near Losne in Burgundy, France, Abbaye..."
2,Abbaye de Tamie,Monks of Tamie,France,Savoie,"8 in. diameter, 2-3 in. high",3 lbs,NaN,Cow,Raw,Semi Soft,Animal,Washed,Washed-Rind,NaN,"Made from raw cow`s milk, Abbaye de Tamie is p..."
3,Abbaye du Mont des Cats,Abbaye du Mont des Cats,France,Nord-Pas-de-Calais,"8 in. diameter, 4 in. high",4 lbs,NaN,Cow,NaN,Semi Soft,Vegetable,Washed,Alpine-styleWashed-Rind,NaN,Abbaye du Mont des Cats is a cow`s milk cheese...
4,Abondance,Various,France,Haut Savoie,"2 ¾ in. tall, 15 ¾ in. diameter",15 - 26 lbs,www.fromageabondance.fr/en/,Cow,Raw,Firm,Animal,Natural,Alpine-style,NaN,Named after a valley in the French Alps where ...
5,Acapella,Andante Dairy,United States,California,Varies,Varies,www.andantedairy.com,Goat,Pasteurised,Soft,Microbial,Mold RipenedAsh coated,Soft-Ripened (Brie-like),NaN,"Established in 1999, Andante Dairy is owned an..."
6,Adams Reserve Cheddar,Great Lakes Cheese,United States,New York,NaN,40 lbs,www.adamsreserve.com,Cow,Pasteurised,Hard,Vegetable,Plasticoat,Cheddar-like,NaN,Adams Reserve is produced by Great Lakes Chees...
7,Adantino with Tarragon,Andante Dairy,United States,California,"3.5 in. long, 1.5 in. wide, 0.75 in. high",5 oz,www.andantedairy.com,Goat,Pasteurised,Soft,Microbial,Mold Ripened,Soft-Ripened (Brie-like),Flavor added to paste,"Established in 1999, Andante Dairy is owned an..."
8,Aged Cheddar (Grafton),Grafton Village Cheese,United States,Vermont,Varies,8 oz; 1 lb; 5 lbs; 10 lbs,http://www.graftonvillagecheese.com/,Cow,Raw,Firm,Microbial,NonePlastic,Cheddar-like,NaN,"Located in Vermont, the Grafton Cooperative Ch..."
9,Airco,Carr Valley Cheese Co,United States,Wisconsin,"8 ins diameter, 4 ins high",10 lbs,www.carrvalleycheese.com,GoatSheepCow,Pasteurised,Semi Soft,Vegetable,Smoked,Alpine-styleCheddar-like,Smoked,Located in the rolling hills of central Wiscon...


In [55]:
dfx = pd.merge(cheese_com, culture_chz, on= 'Name', how ='outer')

In [56]:
dfx

,Name,Milk_x,Country of origin,Region_x,Type,Texture,Rind_x,Colour,Flavour,Aroma,...,Weight,Website,Milk_y,Treatment,Classification,Rennet,Rind_y,Style,Flavor,Description_y
0,Abbaye de Belloc,unpasteurized cow's and sheep's milk,France,Pays Basque,"semi-hard, artisan","creamy, dense and firm",natural,yellow,burnt caramel,lanoline,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Abbaye de Belval,cow's milk,France,NaN,semi-hard,elastic,washed,ivory,NaN,aromatic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Abbaye de Citeaux,unpasteurized cow's milk,France,Burgundy,"semi-soft, artisan, brined","creamy, dense and smooth",washed,white,"acidic, milky, smooth","barnyardy, earthy",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Abbaye de Timadeuc,pasteurized cow's milk,France,province of Brittany,semi-hard,soft,washed,pale yellow,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Abbaye du Mont des Cats,pasteurized cow's milk,France,Nord-Pas-de-Calais,"semi-soft, artisan, brined",smooth and supple,washed,pale yellow,"milky, salty",floral,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Abbot’s Gold,pasteurized cow's milk,"England, Great Britain and United Kingdom",North Yorkshire,semi-hard,"creamy, crumbly, dense and semi firm",natural,pale yellow,"mild, sweet, tangy",aromatic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Abertam,sheep's milk,Czech Republic,Karlovy Vary,"hard, artisan",firm,natural,pale yellow,"acidic, strong, tangy",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Abondance,unpasteurized cow's milk,France,"Haute-Savoie, Abondance","semi-hard, artisan","creamy, open and supple",natural,pale yellow,"acidic, buttery, fruity, sweet",nutty,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Acapella,goat's milk,United States,California,"soft, soft-ripened",NaN,NaN,NaN,buttery,"fresh, herbal",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Accasciato,Buffalo's and cow's milk,Italy,Campania,semi-hard,firm,natural,pale yellow,sweet,"aromatic, fresh",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df3[df3['Name']=='Abbaye de Belloc ']

,Name,Milk_x,Country of origin,Region_x,Type,Texture,Rind_x,Colour,Flavour,Aroma,...,Characteristics,Age,Organic,Category,Animal,Raw_or_Past,Rind,Milk,Region,Description
0,Abbaye de Belloc,unpasteurized cow's and sheep's milk,France,Pays Basque,"semi-hard, artisan","creamy, dense and firm",natural,yellow,burnt caramel,lanoline,...,Uncooked,9 Months,0.0,Firm Cheese,Ewe,Raw Milk,Washed Rind,sheep,FR,From the Pyrenees. Probably the cheese most re...


In [35]:
culture_chz[culture_chz['Name']=='Abbaye de Belloc']

,Name,Producer,Country,Region,Size,Weight,Website,Milk,Treatment,Classification,Rennet,Rind,Style,Flavor,Description
0,Abbaye de Belloc,Abbaye de Notre-Dame de Belloc,France,Aquitaine,"8-9 in. diameter, 3-4 in. high",8-10 lbs,NaN,Sheep,Raw,Firm,Animal,NaturalPaprika,Alpine-styleCheddar-like,Flavor added to rind,Abbaye de Belloc is a French cheese that is ma...


In [51]:
df3[df3['Name']=='Geai Bleu (Le)'].T

""
Name
Milk_x
Country of origin
Region_x
Type
Texture
Rind_x
Colour
Flavour
Aroma


In [70]:
df3.sort_values('Name').reset_index(drop=True)

,Name,Milk_x,Country of origin,Region_x,Type,Texture,Rind_x,Colour,Flavour,Aroma,...,Characteristics,Age,Organic,Category,Animal,Raw_or_Past,Rind,Milk,Region,Description
0,14 Arpents (Le),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Creamy texture, orange rind",1 Month,0.0,Soft Cheese,Cow,Pasteurized,Washed Rind,NaN,NaN,NaN
1,1608 (Le),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3 Months,0.0,Firm Cheese,Cow,Thermised,Washed Rind,NaN,NaN,NaN
2,20 Vaches (Le),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Semi-soft Cheese,Cow,Pasteurized,Washed Rind,NaN,NaN,NaN
3,ADL Brick Cheese,pasteurized cow's milk,Canada,Prince Edward Island,semi-soft,"elastic, firm, open and soft",rindless,ivory,"buttery, mild, milky, subtle","buttery, sweet",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ADL Mild Cheddar,pasteurized cow's milk,Canada,Prince Edward Island,semi-hard,firm and springy,rindless,yellow,"acidic, buttery, milky, subtle",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Abbaye de Belloc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sheep,FR,From the Pyrenees. Probably the cheese most re...
6,Abbaye de Belloc,unpasteurized cow's and sheep's milk,France,Pays Basque,"semi-hard, artisan","creamy, dense and firm",natural,yellow,burnt caramel,lanoline,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Abbaye de Belval,cow's milk,France,NaN,semi-hard,elastic,washed,ivory,NaN,aromatic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Abbaye de Citeaux,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cow/R,FR,Produced by the monks of the Abbey of Saint Ni...
9,Abbaye de Citeaux,unpasteurized cow's milk,France,Burgundy,"semi-soft, artisan, brined","creamy, dense and smooth",washed,white,"acidic, milky, smooth","barnyardy, earthy",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
